## 1. Setup Confluent Kafka (credentials & test connection)

In [0]:
# Get confluent kafka secrets

bootstrap_servers = dbutils.secrets.get(scope="confluent-kafka", key="bootstrap-servers")
sasl_username = dbutils.secrets.get(scope="confluent-kafka", key="api-key")
sasl_password = dbutils.secrets.get(scope="confluent-kafka", key="api-secret")


In [0]:
# Other configurations
topic = "orders"
bronze_table = "bronze.orders_stream"
bronze_checkpoint = "/Volumes/workspace/bronze/checkpoint"

In [0]:
# Check connectivity

try:

    jaas_config = "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(sasl_username, sasl_password)

    test_df = (spark.readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", bootstrap_servers)
        .option("kafka.sasl.mechanism", "PLAIN")
        .option("kafka.security.protocol", "SASL_SSL")
        .option("kafka.sasl.jaas.config", jaas_config)
        .option("subscribe", topic)
        .option("startingOffsets", "earliest")
        .load()
    )

    display(test_df, checkpointLocation = bronze_checkpoint)

except Exception as e:
    print( f"Connection failed: {e}")